In [10]:
import zoom_api
from pathlib import Path

## Download Participants

### Get Zoom Authorization Token

Note: Don't print

In [8]:
creds = zoom_api.Credentials.from_environment_variables()
oauth = zoom_api.get_server2server_oauth(
    account_id=creds.zoom_account_id,
    client_id=creds.zoom_client_id,
    client_secret=creds.zoom_client_secret,
)
token = oauth['access_token']
assert token

### Get Parts for Meetings with Registration

In [11]:
from snakemake.io import glob_wildcards

In [13]:
meeting_ids, = glob_wildcards('data/registrations/registrations_for_meeting_{meeting_id}.json')
meeting_ids

Need the uuid for the meeting, so let's go searching for it.

In [ ]:
def list_participants(token, from_date, to_date) -> Iterator[pd.DataFrame]:
    user_df = pd.DataFrame(zoom_api.list_users(token=token))
    owner_id = user_df[user_df.role_id == '0'].iloc[-1].id
    meetings = zoom_api.list_past_meeting_sessions(token=token, user_id=owner_id, from_=from_date, to=to_date)

    for meeting in meetings:
        events = pd.DataFrame(zoom_api.get_meeting_participant_events(token=token, meeting_id=meeting['uuid']))
        for key, value in meeting.items():
            events['meeting_' + key] = value
        yield events

In [9]:
list_participants(
    token=token, 
    from_date='2022-10-24', 
    to_date=datetime.today().strftime("%Y-%m-%d"),
)

In [ ]:
['tok']